In [238]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [239]:
product_type = "frozen"
num_time_periods = 7

In [240]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [241]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 50

In [242]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [243]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [244]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [245]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [246]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [247]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [248]:
# test_article = '00ee8964'
# for t in demand.keys():
#     demand[t][test_article] = 0
# demand[3][test_article] = 5
# shelf_life[test_article] = 1
# cu_per_tu[test_article] = 1
# # warehouse_volume = 1

In [249]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}
        self.clearance_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [250]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0
    for item in cell.orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            # CHECK
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
            # CHECK
            volume += fetchExtraVolume(cell, t)
        else:
            break
    
    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1) 

order_date = 0
end_reached = False
obj_val = 0
while(True):
    min_cost = float('inf')
    for t in range(order_date,n):
        getCost(cost_matrix[order_date][t])
        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            if(t==n-1):
                end_reached = True
                break
        else:
            order_date = t
            break
    obj_val += cost_matrix[order_date][t].cost
    if(order_date >= n or end_reached):
        break

obj_val

6616.199999999997

In [251]:
def getSchedule():
    schedule = {}
    for i in range(len(cost_matrix)):
        for j in range(len(cost_matrix[i])):
            cell = cost_matrix[i][j]
            if(type(cell)==CostMatrixCell):

                if(cell.order_date not in schedule.keys()):
                    schedule[cell.order_date] = {}

                for item in cell.orders.keys():
                    if(item not in schedule[cell.order_date].keys()):
                        schedule[cell.order_date][item] = 0
                    schedule[cell.order_date][item] += cell.orders[item]

                for extra_date in cell.extra_orders.keys():
                    if(extra_date not in schedule.keys()):
                        schedule[extra_date] = {}
                    
                    for extra_item in cell.extra_orders[extra_date].keys():
                        if(extra_item not in schedule[extra_date].keys()):
                            schedule[extra_date][extra_item] = 0
                        schedule[extra_date][extra_item] += cell.extra_orders[extra_date][extra_item]

    return schedule

print(getSchedule())

{0: {'00ee8964': 19, '010c48db': 17, '015eb098': 14, '019f5beb': 83, '020b50b0': 83, '03436da5': 19, '038bb02c': 13, '04112e43': 17, '04828ea6': 23, '050daac5': 13, '05ffd974': 221, '0744ee79': 14, '0793a340': 22, '097bbc43': 245, '0983d839': 32, '09e397f8': 117, '0b1a45e0': 13, '0ba6590b': 43, '0e611aba': 14, '1017897f': 466, '105e43d9': 24, '106ac8f0': 419, '10f2b453': 62, '111b2434': 14, '11545bca': 49, '126334a5': 29, '12c460b7': 39, '12cf1bca': 55, '13541e66': 223, '1377878b': 50, '138278a7': 9, '13becd5f': 22, '148ed136': 20, '15b0e7a5': 16, '17f9fecb': 18, '1814bccd': 10, '189067f1': 56, '196c247b': 59, '1a30c679': 26, '1c939a8e': 32, '1c96fbe0': 13, '1ccd89f6': 41, '1d34848b': 67, '1d4d730b': 25, '1e6e2355': 41, '1f15ae34': 13, '1f782edd': 13, '1f9d7d64': 50, '227e7261': 40, '230872dd': 291, '2410d366': 17, '2423482a': 13, '25b50b2a': 62, '26fa73fc': 20, '27c511b5': 17, '2a71fdad': 13, '2ab442df': 162, '2af749f6': 13, '2b6aadc0': 31, '2b950121': 31, '2cd380af': 21, '2d54eaba': 